In [9]:
import zarr
import xarray as xr
import numpy as np
import time
import math


In [43]:
st = time.time()
ds = xr.open_zarr(
    'GEBCO_2022_sub_ice_topo.zarr', chunks='auto', 
    group='gebco', decode_cf=False, decode_times=False)

et = time.time()
print('Exe time: ', et-st, 'sec')
ds

Exe time:  0.005028247833251953 sec


<xarray.Dataset>
Dimensions:    (lat: 43200, lon: 86400)
Coordinates:
  * lat        (lat) float64 -90.0 -89.99 -89.99 -89.99 ... 89.99 89.99 90.0
  * lon        (lon) float64 -180.0 -180.0 -180.0 -180.0 ... 180.0 180.0 180.0
Data variables:
    crs        |S1 ...
    elevation  (lat, lon) int16 dask.array<chunksize=(675, 2700), meta=np.ndarray>
Attributes: (12/36)
    Conventions:                     CF-1.6, ACDD-1.3
    comment:                         The data in the GEBCO_2022 Grid should n...
    creator_email:                   gdacc@seabed2030.org
    creator_name:                    GEBCO through the Nippon Foundation-GEBC...
    creator_type:                    International organisation
    creator_url:                     https://www.gebco.net
    ...                              ...
    node_offset:                     1.0
    project:                         Nippon Foundation - GEBCO Seabed2030 Pro...
    references:                      DOI: 10.5285/e0f0bb80-ab44-2739-e053-6c8...
    source:                          The GEBCO_2022 Grid is the latest global...
    summary:                         The GEBCO_2022 Grid is a continuous, glo...
    title:                           The GEBCO_2022 Grid - a continuous terra...

In [35]:
# base=90 for latitude; base=180 for longitude
def gridded_arcsec(x, base=90, arc=3600/15):
    return ((int(x) + base)* arc + math.ceil((x-int(x))*arc))

arc = int(3600/15) #arc
print(gridded_arcsec(0, 180, arc))
print(gridded_arcsec(0, 90, arc))
print(gridded_arcsec(-90, 90, arc))
print(gridded_arcsec(90, 90, arc))
print(gridded_arcsec(-180, 180, arc))
print(gridded_arcsec(180, 180, arc))
print(gridded_arcsec(0.0015, 180, arc))
print(gridded_arcsec(-0.00207, 180, arc))
print(gridded_arcsec(-85.375, 90, arc))
print(gridded_arcsec(48.37918, 90, arc))


43200
21600
0
43200
0
86400
43201
43200
1110
33212


In [54]:
arcsec = 15
arc = int(3600/arcsec) #15 arc-second
basex = 180 #-180 - 180 <==> 0 - 360, half is 180
basey = 90  #-90 - 90 <==> 0 - 180, half is 90
halfxidx = 180 * arc   #in netcdf, longitude length = 86400
halfyidx = 90 * arc    #in netcdf, latitude length = 43200
lon = [123.98, 123]
lat = [23.33, 21.33]
subsetFlag = True
if (len(lon) != len(lat)):
    print("Input Error: check longitude, latitude length not equal")
    #return
# May not have exact lon, lat in gridded lon-lat, so need to calculate index  
elif (len(lon) == 1):
    st = time.time()
    lon0 = gridded_arcsec(lon[0], basex, arc)
    lat0 = gridded_arcsec(lat[0], basey, arc)
    st1 = ds.isel(lon=lon0, lat=lat0)
    xt1 = [st1['elevation'].values]
    et = time.time()
    print('Exe time: ', et-st, 'sec')
    print(xt1)
else:
    st = time.time()
    idx1 = np.empty(shape=[0, 2], dtype=np.int16)
    loc1 = np.empty(shape=[0, 2], dtype=float)
    ele1 = np.empty(shape=[0, 1], dtype=np.int16)

    mlon0 = np.min(lon)
    #mlon1 = np.max(lon)
    mlat0 = np.min(lat)
    #mlat1 = np.max(lat)
    #ds_s1 = ds.sel(lon=slice(mlon0, mlon1), lat=slice(mlat0, mlat1))
    mlonbase = gridded_arcsec(mlon0, basex, arc) if subsetFlag else 0 #if do subsetting dataset, reference-0-x,y should be biased
    mlatbase = gridded_arcsec(mlat0, basey, arc) if subsetFlag else 0
    print(mlonbase, mlatbase)
    #mlonidx1 = gridded_arcsec(mlon1, basex, arc)
    #mlatidx1 = gridded_arcsec(mlat1, basey, arc)
    for i in range(len(lon)-1):
        lonidx0 = gridded_arcsec(lon[i], basex, arc)
        latidx0 = gridded_arcsec(lat[i], basey, arc)
        lonidx1 = gridded_arcsec(lon[i+1], basex, arc)
        latidx1 = gridded_arcsec(lat[i+1], basey, arc)
        #ds_s1 = ds.sel(lon=slice(lon[i], lon[i+1]), lat=slice(lat[i], lat[i+1]))
        #index from gridded_arcsec will change is reference is ds_s1, not ds
        if (lonidx0 == lonidx1 and latidx0 == latidx1):
            idx1 = np.append(idx1, [[latidx0-mlatbase, lonidx0-mlonbase]], axis=0)
            loc1 = np.append(loc1, [[lat[i], lon[i]]], axis=0)
        elif(lonidx0 == lonidx1):
            stepi = -1 if latidx0 > latidx1 else 1
            for y in range(latidx0, latidx1, stepi):
                idx1 = np.append(idx1, [[y-mlatbase, lonidx0-mlonbase]], axis=0)
                locy0 = y/arc - basey
                locy0i= int(locy0)
                doty0i= locy0-locy0i-0.25/arc #a small bias to make sure it's in grid
                print("yi: ", y, locy0i + doty0i)
                loc1 = np.append(loc1, [[locy0i + doty0i, lon[i]]], axis=0)
        elif(latidx0 == latidx1):
            stepi = -1 if lonidx0 > lonidx1 else 1
            for x in range(lonidx0, lonidx1, stepi):
                idx1 = np.append(idx1, [[latidx0-mlatbase, x-mlonbase]], axis=0)
                locx0 = x/arc - basex
                locx0i= int(locx0)
                dotx0i= locx0-locx0i-0.25/arc #a small bias to make sure it's in grid
                print("xi: ", x, locx0i + dotx0i)
                loc1 = np.append(loc1, [[lat[i], locx0i + dotx0i]], axis=0)
        else:
            m = (lat[i+1]-lat[i])/(lon[i+1]-lon[i])
            b = lat[i] - m * lon[i] #y = mx + b
            print("m, b:", m ,b)
            stepi = -1 if lonidx0 > lonidx1 else 1
            for k, x in enumerate(range(lonidx0, lonidx1, stepi)):
                if (k==0): #should consider internal node not repeated twice
                    idx1 = np.append(idx1, [[latidx0-mlatbase, lonidx0-mlonbase]], axis=0)
                    loc1 = np.append(loc1, [[lat[i], lon[i]]], axis=0)
                else:
                    locx0 = x/arc - basex
                    locx0i= int(locx0)
                    dotx0i= locx0-locx0i-0.25/arc #a small bias to make sure it's in grid
                    locx1 = locx0i + dotx0i 
                    locy1 = m * locx1 + b 
                    y = gridded_arcsec(locy1, basey, arc)
                    idx1 = np.append(idx1, [[y-mlatbase, x-mlonbase]], axis=0)
                    loc1 = np.append(loc1, [[locy1, locx1]], axis=0)
                    #print(x, locx0, dotx0i, locx1, locy1, y)
                    
    et = time.time()
    print('Exe time: ', et-st, 'sec')
    print(idx1)
    np.savetxt("simu/test_loc.csv", loc1, delimiter=",", fmt='%f')

    




0 0
m, b: 2.040816326530604 -229.6904081632643
Exe time:  0.004813671112060547 sec
[[27200 72956]
 [27199 72955]
 [27197 72954]
 [27195 72953]
 [27193 72952]
 [27191 72951]
 [27189 72950]
 [27187 72949]
 [27184 72948]
 [27182 72947]
 [27180 72946]
 [27178 72945]
 [27176 72944]
 [27174 72943]
 [27172 72942]
 [27170 72941]
 [27168 72940]
 [27166 72939]
 [27164 72938]
 [27162 72937]
 [27160 72936]
 [27158 72935]
 [27156 72934]
 [27154 72933]
 [27152 72932]
 [27150 72931]
 [27148 72930]
 [27146 72929]
 [27144 72928]
 [27142 72927]
 [27140 72926]
 [27138 72925]
 [27136 72924]
 [27133 72923]
 [27131 72922]
 [27129 72921]
 [27127 72920]
 [27125 72919]
 [27123 72918]
 [27121 72917]
 [27119 72916]
 [27117 72915]
 [27115 72914]
 [27113 72913]
 [27111 72912]
 [27109 72911]
 [27107 72910]
 [27105 72909]
 [27103 72908]
 [27101 72907]
 [27099 72906]
 [27097 72905]
 [27095 72904]
 [27093 72903]
 [27091 72902]
 [27089 72901]
 [27087 72900]
 [27084 72899]
 [27082 72898]
 [27080 72897]
 [27078 72896]
 [

In [56]:
ds_s1 = ds.sel(lon=slice(mlon0, mlon1), lat=slice(mlat0, mlat1)) 
print(ds_s1)
print(ds["lat"][26720]) #actually more near 21.34, not 21.33 so slice 21.33 makes the offset 1 possibly!!
print(ds["lon"][72720]) #should use print(ds["lat"][26720]) to calcuate back
print(ds["elevation"].values[26720,72720]) 

<xarray.Dataset>
Dimensions:    (lat: 482, lon: 237)
Coordinates:
  * lat        (lat) float64 21.33 21.34 21.34 21.34 ... 23.32 23.33 23.33 23.34
  * lon        (lon) float64 123.0 123.0 123.0 123.0 ... 124.0 124.0 124.0 124.0
Data variables:
    crs        |S1 ...
    elevation  (lat, lon) int16 dask.array<chunksize=(281, 180), meta=np.ndarray>
Attributes: (12/36)
    Conventions:                     CF-1.6, ACDD-1.3
    comment:                         The data in the GEBCO_2022 Grid should n...
    creator_email:                   gdacc@seabed2030.org
    creator_name:                    GEBCO through the Nippon Foundation-GEBC...
    creator_type:                    International organisation
    creator_url:                     https://www.gebco.net
    ...                              ...
    node_offset:                     1.0
    project:                         Nippon Foundation - GEBCO Seabed2030 Pro...
    references:                      DOI: 10.5285/e0f0bb80-ab44-2739-e0

In [57]:
st = time.time()
#mlon0 = np.min(lon) #may cause slice offset to mlonbase, an offset +-1
mlon1 = np.max(lon) + 1.5/arc #to make it larger
#mlat0 = np.min(lat) #may cause slice offset to mlatbase, an offset +-1
mlat1 = np.max(lat) + 1.5/arc
mlon0x= ds["lon"][mlonbase]
mlat0x= ds["lat"][mlatbase]
ds_s1 = ds.sel(lon=slice(mlon0x, mlon1), lat=slice(mlat0x, mlat1)) if subsetFlag else ds
print(ds_s1['elevation'].shape)
#mlonidx0 = gridded_arcsec(mlon0, basex, arc)
#mlatidx0 = gridded_arcsec(mlat0, basey, arc)
#mlonidx1 = gridded_arcsec(mlon1, basex, arc)
#mlatidx1 = gridded_arcsec(mlat1, basey, arc)
loct = tuple(idx1.T)
print(loct)
pt1=ds_s1['elevation'].values[loct]
et = time.time()
print('Get index of points, time: ', et-st, 'sec')
print(pt1)
np.savetxt("simu/test_zseg.csv", pt1, delimiter=",", fmt='%f')

(43200, 86400)
(array([27200, 27199, 27197, 27195, 27193, 27191, 27189, 27187, 27184,
       27182, 27180, 27178, 27176, 27174, 27172, 27170, 27168, 27166,
       27164, 27162, 27160, 27158, 27156, 27154, 27152, 27150, 27148,
       27146, 27144, 27142, 27140, 27138, 27136, 27133, 27131, 27129,
       27127, 27125, 27123, 27121, 27119, 27117, 27115, 27113, 27111,
       27109, 27107, 27105, 27103, 27101, 27099, 27097, 27095, 27093,
       27091, 27089, 27087, 27084, 27082, 27080, 27078, 27076, 27074,
       27072, 27070, 27068, 27066, 27064, 27062, 27060, 27058, 27056,
       27054, 27052, 27050, 27048, 27046, 27044, 27042, 27040, 27038,
       27036, 27033, 27031, 27029, 27027, 27025, 27023, 27021, 27019,
       27017, 27015, 27013, 27011, 27009, 27007, 27005, 27003, 27001,
       26999, 26997, 26995, 26993, 26991, 26989, 26987, 26984, 26982,
       26980, 26978, 26976, 26974, 26972, 26970, 26968, 26966, 26964,
       26962, 26960, 26958, 26956, 26954, 26952, 26950, 26948, 26946,
    

In [65]:
from fastapi.encoders import jsonable_encoder
#from fastapi.responses import JSONResponse
json_data = jsonable_encoder({"data": pt1.tolist()})
print(json_data)

{'data': [-3729, -3819, -4168, -4401, -4703, -4904, -5270, -5406, -5315, -5342, -5383, -5387, -4946, -4911, -5304, -5683, -5642, -5557, -5631, -5698, -5646, -5634, -5733, -5767, -5900, -6040, -6124, -6110, -6134, -6234, -6300, -6350, -6357, -6422, -6399, -6360, -6364, -6362, -6362, -6373, -6372, -6333, -6358, -6348, -6346, -6361, -6346, -6335, -6327, -6316, -6330, -6319, -6326, -6336, -6361, -6351, -6299, -6295, -6273, -6267, -6266, -6270, -6264, -6256, -6247, -6225, -6196, -6126, -6100, -6102, -6072, -6060, -6035, -5971, -5988, -5984, -5917, -5797, -5655, -5581, -5492, -5514, -5544, -5569, -5587, -5598, -5611, -5612, -5632, -5618, -5604, -5585, -5564, -5548, -5525, -5528, -5521, -5546, -5568, -5610, -5636, -5642, -5639, -5637, -5648, -5670, -5705, -5733, -5750, -5756, -5757, -5751, -5746, -5743, -5729, -5702, -5602, -5736, -5706, -5709, -5721, -5710, -5700, -5666, -5620, -5585, -5584, -5528, -5499, -5479, -5328, -5421, -5439, -5418, -5419, -5302, -5307, -5379, -5617, -5624, -5641, -56